<b> Goal: Generate Generic code that merges/joins data from csvs into dataframe

In [1]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None 

In [2]:
# Forecast Data
## Reading data and formating data-time columns
df_forecast = pd.read_csv('data/forecastdemand_nsw.csv', names = ['id', 'region_id', 'period_id', 'forecast_demand', 'date_time_current', 'date_time_future'], skiprows = 1)
df_forecast.date_time_current = pd.to_datetime(df_forecast.date_time_current, format = "%Y-%m-%d %H:%M:%S")
df_forecast.date_time_future = pd.to_datetime(df_forecast.date_time_future, format = "%Y-%m-%d %H:%M:%S")

## Using 'period_id' to round 'current timee'
df_forecast["date_time_current_rounded"] = df_forecast.period_id.apply(lambda x: pd.Timedelta(hours = x/2))
df_forecast.date_time_current_rounded = df_forecast.date_time_future - df_forecast.date_time_current_rounded

In [3]:
# Demand Data
## Reading data and formating data-time columns
df_demand = pd.read_csv('data/totaldemand_nsw.csv', names = ['date_time', 'total_demand', 'region_id'], skiprows = 1)
df_demand.date_time = pd.to_datetime(df_demand.date_time, format = "%d/%m/%Y %H:%M")

In [4]:
# Temperature Data
## Reading data and formating data-time columns
df_temperature = pd.read_csv('data/temperature_nsw.csv', names = ['location', 'date_time', 'temperature'], skiprows = 1)
df_temperature.date_time = pd.to_datetime(df_temperature.date_time, format = "%d/%m/%Y %H:%M")

# Setting time intervals between data as 30minutes
df_temperature["date_time_30m"] = df_temperature.date_time.dt.round('30T')
df_temperature["date_time_30m_interval"] = abs(df_temperature.date_time - df_temperature.date_time_30m)
df_temperature = df_temperature.loc[df_temperature.groupby("date_time_30m")["date_time_30m_interval"].idxmin()]

In [5]:
# Forecast temperature data
df_weather_forecast = pd.read_csv('data/forecast_temperatre.csv')

df_weather_forecast = df_weather_forecast.rename({'forecast dt iso': 'date_time_current_utc', 
                                                  'slice dt iso': 'date_time_future_utc',
                                                  'temperature': 'temperature_future_forecast',
                                                  'humidity': 'humidity_future_forecast',
                                                  'dew_point': 'dew_point_future_forecast',
                                                  'wind_speed': 'wind_speed_future_forecast'}, axis = 1)

df_weather_forecast["date_time_current_rounded"] = pd.to_datetime(df_weather_forecast.date_time_current_utc, format = "%Y-%m-%d %H:%M:%S +0000 UTC") + pd.Timedelta(hours = 10)
df_weather_forecast["date_time_future"] = pd.to_datetime(df_weather_forecast.date_time_future_utc, format = "%Y-%m-%d %H:%M:%S +0000 UTC") + pd.Timedelta(hours = 10)

df_weather_forecast = df_weather_forecast[['date_time_current_rounded', 'date_time_future', 'temperature_future_forecast', 'humidity_future_forecast', 'dew_point_future_forecast', 'wind_speed_future_forecast']]

In [6]:
#Merging Datasets
df_all = pd.merge(df_forecast, df_demand[["date_time", "total_demand"]], left_on = "date_time_future", right_on = "date_time").drop(columns = "date_time")

df_all = pd.merge(df_all, df_temperature[["date_time_30m", "temperature"]], left_on = "date_time_future", right_on = "date_time_30m")
df_all = df_all.drop(columns = ["date_time_30m", "region_id"]).rename({"temperature": "temperature_future"}, axis = 1)

df_all = pd.merge(df_all, df_temperature[["date_time_30m", "temperature"]], left_on = "date_time_current_rounded", right_on = "date_time_30m")
df_all = df_all.drop(columns = "date_time_30m").rename({"temperature": "temperature_current"}, axis = 1)

df_all = pd.merge(df_all, df_weather_forecast, on = ["date_time_current_rounded", "date_time_future"], how = 'left')

df_all["forecast_interval"] = df_all.date_time_future - df_all.date_time_current_rounded

In [7]:
df_all.to_csv("data/combined_data.csv", index = False)

In [8]:
df_all.loc[df_all.temperature_future_forecast.notna()].sample(10)

,id,period_id,forecast_demand,date_time_current,date_time_future,date_time_current_rounded,total_demand,temperature_future,temperature_current,temperature_future_forecast,humidity_future_forecast,dew_point_future_forecast,wind_speed_future_forecast,forecast_interval
10834054,2021030937,42,8679.12,2021-03-09 22:01:43,2021-03-10 19:00:00,2021-03-09 22:00:00,8589.68,22.4,21.9,20.59,69.30,14.80,3.51,0 days 21:00:00
9927842,2020040325,62,6711.89,2020-04-03 16:01:27,2020-04-04 23:00:00,2020-04-03 16:00:00,6567.71,14.3,23.3,14.63,60.09,6.84,1.21,1 days 07:00:00
10046976,2020051813,18,9745.08,2020-05-18 10:00:55,2020-05-18 19:00:00,2020-05-18 10:00:00,9833.52,15.6,16.3,14.45,71.70,9.35,0.66,0 days 09:00:00
9941110,2020040825,10,7812.37,2020-04-08 16:01:40,2020-04-08 21:00:00,2020-04-08 16:00:00,7491.51,17.5,18.5,15.16,88.25,13.18,2.16,0 days 05:00:00
8637337,2018120601,18,7833.57,2018-12-06 04:01:06,2018-12-06 13:00:00,2018-12-06 04:00:00,8057.17,24.9,17.3,24.37,47.95,12.73,5.32,0 days 09:00:00
9136007,2019061113,10,8010.47,2019-06-11 10:00:59,2019-06-11 15:00:00,2019-06-11 10:00:00,7713.21,20.2,17.5,20.11,52.45,10.05,3.25,0 days 05:00:00
8995579,2019041925,72,5674.61,2019-04-19 16:01:38,2019-04-21 04:00:00,2019-04-19 16:00:00,5706.97,15.7,23.3,17.86,94.80,16.95,1.29,1 days 12:00:00
10826680,2021030701,6,6315.22,2021-03-07 04:01:30,2021-03-07 07:00:00,2021-03-07 04:00:00,6317.82,15.9,15.9,17.12,77.80,13.19,0.94,0 days 03:00:00
8172468,2018061425,64,8550.03,2018-06-14 16:01:58,2018-06-16 00:00:00,2018-06-14 16:00:00,8621.62,7.7,17.9,10.63,55.71,2.06,2.64,1 days 08:00:00
9047591,2019050901,16,7625.93,2019-05-09 04:01:02,2019-05-09 12:00:00,2019-05-09 04:00:00,7523.13,19.4,7.9,19.00,34.74,3.25,2.48,0 days 08:00:00


In [9]:
df_weather_forecast

,date_time_current_rounded,date_time_future,temperature_future_forecast,humidity_future_forecast,dew_point_future_forecast,wind_speed_future_forecast
0,2017-10-07 10:00:00,2017-10-07 10:00:00,19.75,45.60,7.75,2.57
1,2017-10-07 10:00:00,2017-10-07 11:00:00,20.81,41.98,7.33,3.07
2,2017-10-07 10:00:00,2017-10-07 12:00:00,21.38,40.79,7.48,3.59
3,2017-10-07 10:00:00,2017-10-07 13:00:00,21.50,41.81,8.05,4.07
4,2017-10-07 10:00:00,2017-10-07 14:00:00,21.20,44.79,8.85,4.43
...,...,...,...,...,...,...
3882074,2025-03-31 04:00:00,2025-04-16 00:00:00,18.87,61.28,11.25,2.18
3882075,2025-03-31 04:00:00,2025-04-16 01:00:00,18.20,62.60,10.92,1.94
3882076,2025-03-31 04:00:00,2025-04-16 02:00:00,17.67,63.94,10.74,1.84
3882077,2025-03-31 04:00:00,2025-04-16 03:00:00,17.38,64.81,10.66,1.95
